# <center> Predicting Donors Choose</center>

--------

# Introduction

## Imports

https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html

In [157]:
import gc
import numpy as np
import pandas as pd
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import tqdm

In [158]:
import lightgbm as lgb

In [159]:
"""
import gensim

from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
"""
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import re

In [160]:
desk = True

In [161]:
train = pd.read_csv('./Input/train.csv', low_memory=False, index_col='id')
if desk: test = pd.read_csv('./Input/test.csv', low_memory=False, index_col='id')

res = pd.read_csv('./Input/resources.csv', low_memory=False, index_col='id')

In [162]:
train = train[:500]
test = test[:500]

## Preprocessing

### 1.1 Resource Intergration
Here we evaluate how much each project/proposal will cost and/or how big they are

In [163]:
res['cost'] = res['quantity'] * res['price']
res_agg = res.groupby('id').agg({'description': ['nunique'], 'quantity': ['sum'], 'cost': ['mean', 'sum']})
res_agg.columns = ['unique_items', 'total_quantity', 'mean_cost', 'total_cost']
res_agg.reset_index(inplace=True)

#description was dropped because the description of the project should not have an effect on its liklihood of success

train = train.merge(res_agg, left_index=True, right_on='id')
if desk: test =  test.merge(res_agg, left_index=True, right_on='id')

del res_agg
del res

### 1.3 Preprocessing of features

#### Cat Preprocessing
- Improve states
    - States are 51 because of 50 + DC

In [164]:
train.isnull().sum()

teacher_id                                        0
teacher_prefix                                    0
school_state                                      0
project_submitted_datetime                        0
project_grade_category                            0
project_subject_categories                        0
project_subject_subcategories                     0
project_title                                     0
project_essay_1                                   0
project_essay_2                                   0
project_essay_3                                 482
project_essay_4                                 482
project_resource_summary                          0
teacher_number_of_previously_posted_projects      0
project_is_approved                               0
id                                                0
unique_items                                      0
total_quantity                                    0
mean_cost                                         0
total_cost  

In [165]:
train.teacher_prefix[train.teacher_prefix.isnull()] = 'Teacher'
if desk: test.teacher_prefix[test.teacher_prefix.isnull()] = 'Teacher'

/Users/Nahel/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Nahel/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


fill na did not work

In [166]:
def date_prep(train):
    train['project_submitted_datetime'] = pd.to_datetime(train['project_submitted_datetime'])
    train['datetime_year'] = train['project_submitted_datetime'].dt.year
    train['datetime_month'] = train['project_submitted_datetime'].dt.month

    del train['project_submitted_datetime']
    del train['project_subject_subcategories']
    return(train)

date_prep(train)
if desk: date_prep(test)

In [167]:
def gender_features(train):
    conditions = [(train.teacher_prefix == 'Mr.'), 
                  (train.teacher_prefix == 'Mrs.') | (train.teacher_prefix == 'Ms.')]
    choices = ['Male', 'Female']
    train['gender'] = np.select(conditions, choices, default='Unk')
    return(train)

In [168]:
train = gender_features(train)
if desk: test = gender_features(test)
del gender_features

In [169]:
#train.groupby(['datetime_month','gender'])['project_is_approved'].mean().reset_index()

In [170]:
#gen_mon.to_csv('gb_gender_month.csv')

In [171]:
#sns.lmplot(x="datetime_month", y="project_is_approved", hue="gender", data=gen_mon[gen_mon['gender']!='Unk'], order=4, ci=None, truncate=True)

In [172]:
gc.collect()

31

##### Encoding labels

In [173]:
train = train.reset_index()
if desk: train = train.reset_index()

In [174]:
# Because of memory issues, it made more sense to encode everything as a string rather than dummies
cols  = [
    'teacher_id', 
    'gender',
#    'datetime_year', already encoded
#    'datetime_month', already encoded
    'teacher_prefix', 
    'school_state', 
    'project_grade_category',
    'project_subject_categories']

for c in tqdm.tqdm_notebook(cols):
    encod = LabelEncoder()
    encod.fit(train[c].astype(str))
    train[c] = encod.transform(train[c].astype(str))

if desk: 
    for c in tqdm.tqdm_notebook(cols):
        encod = LabelEncoder()
        encod.fit(test[c].astype(str))
        test[c] = encod.transform(test[c].astype(str))

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

In [175]:
del cols
del encod
del LabelEncoder
gc.collect()

14

#### Num Preprocessing

In [176]:
num_features  = ['teacher_number_of_previously_posted_projects','total_quantity', 'mean_cost', 'total_cost',]

In [177]:
SS = StandardScaler()
train[num_features] = SS.fit_transform(train[num_features])
if desk: test[num_features] = SS.fit_transform(test[num_features])

In [178]:
del num_features
del StandardScaler
del SS

In [179]:
gc.collect()

14

#### Text Preprocessing

  
#### Before May 17th, 2016:

- project_essay_1: "Introduce us to your classroom"
- project_essay_2: "Tell us more about your students"
- project_essay_3: "Describe how your students will use the materials you're requesting"
- project_essay_4: "Close by sharing why your project will make a difference"

#### May 17th, 2016 and beyond:

- project_essay_1: "Describe your students: What makes your students special? Specific details about their background, your neighborhood, and your school are all helpful."
- project_essay_2: "About your project: How will these materials make a difference in your students' learning and improve their school lives?"

#### Plan
- Combine essay_1 and essay_2 before May 17th to make "student_description" and use essay_1 after May 17th directly
- Combine essay_3 and essay_4 before May 17th to make "project_description" and use essay_2 after May 17th directly

In [180]:
def essay_convert(train):
    # Making the First essay : student_description
    train['student_description']=train['project_essay_1']
    train.loc[train.project_essay_3.notnull(),'student_description']=train.loc[train.project_essay_3.notnull(),'project_essay_1']+train.loc[train.project_essay_3.notnull(),'project_essay_2']

    # Making the second essay : project_description
    train['project_description']=train['project_essay_2']
    train.loc[train.project_essay_3.notnull(),'project_description']=train.loc[train.project_essay_3.notnull(),'project_essay_3']+train.loc[train.project_essay_3.notnull(),'project_essay_4']

    # Removing original essays
    del train['project_essay_1']
    del train['project_essay_2']
    del train['project_essay_3']
    del train['project_essay_4']
    return(train)

In [181]:
essay_convert(train)
if desk: essay_convert(test)
gc.collect()
del essay_convert

### Lem & Tokenizer

In [182]:
text_features = ['project_title', 'project_resource_summary',
                'project_description', 'student_description']

In [183]:
other_stopwords = [x for x in 'abcdefghijklmnopqrstuvwxyz']
for j in ['student','students','education',]:
    other_stopwords.append(j)

In [184]:
#import gensim
from nltk.corpus import stopwords
def scrub(text):
    text = text.strip().lower()
    text = re.sub('\W+',' ', text)
    text = re.sub(r'(\")', ' ', text)
    text = re.sub(r'(\r)', ' ', text)
    text = re.sub(r'(\n)', ' ', text)
    text = re.sub(r'(\r\n)', ' ', text)
    text = re.sub(r'(\\)', ' ', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\:', ' ', text)
    text = re.sub(r'\"\"\"\"', ' ', text)
    text = re.sub(r'_', ' ', text)
    text = re.sub(r'\+', ' ', text)
    text = re.sub(r'\=', ' ', text)
    text = re.sub(' i m ',' i\'m ', text)
    text = re.sub('n t ','n\'t ', text)
    text = re.sub(' re ',' are ', text)
    text = ' '.join([word for word in text.split() if word not in (other_stopwords + stopwords.words("english"))])
    return(text)

In [185]:
gc.collect()

0

In [186]:
for j in tqdm.tqdm_notebook(text_features):
    n_col = 'processed_'+j
    train[n_col] = train[j].apply(lambda x: scrub(x))
    if desk: test[n_col] = test[j].apply(lambda x: scrub(x))

gc.collect()
for i in text_features:
    del train[i]
    if desk: del test[i]

del stopwords
del other_stopwords
del text_features

HBox(children=(IntProgress(value=0, max=4), HTML(value='')))

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/Users/Nahel/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/Nahel/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/Nahel/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



## Modeling

### Non_NN

#### Tfidf & X,y Assignment

In [187]:
gc.collect()

56

In [188]:
cols = [
    'processed_project_title',
    'processed_project_resource_summary', 
    'processed_project_description',
    'processed_student_description']

In [189]:
for i in cols:
    print("Average length in {} is {} words".format(i,str(round(train[i].str.len().mean()))))

Average length in processed_project_title is 24 words
Average length in processed_project_resource_summary is 80 words
Average length in processed_project_description is 513 words
Average length in processed_student_description is 402 words


In [190]:
n_features = [
    200, 
    400, 
    2500,
    2500]

In [191]:
gc.collect()

122

In [194]:
for c_i, c in tqdm.tqdm_notebook(enumerate(cols)):
    tfidf = TfidfVectorizer(
        max_features=n_features[c_i])

    tfidf.fit(train[c])

    tfidf_train2 = tfidf.transform(train[c])

#    tfidf_train = np.array(tfidf.transform(train[c].values).toarray(), dtype=np.float16)
#    for i in range(n_features[c_i]):
#        train[c + '_tfidf_' + str(i)] = tfidf_train[:, i]
#    if desk:
#        tfidf_test = np.array(tfidf.transform(test[c].values).toarray(), dtype=np.float16)
#        for i in range(n_features[c_i]):
#            test[c + '_tfidf_' + str(i)] = tfidf_test[:, i]

#for i in cols:
#    del train[i]
#    if desk: del test[i]

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [198]:
tfidf_train2

<500x2500 sparse matrix of type '<class 'numpy.float64'>'
	with 22653 stored elements in Compressed Sparse Row format>

In [123]:
y = train['project_is_approved']
y = y.copy(deep=True)

In [124]:
drop_cols = ['index','project_is_approved','unique_items']
del train['id']

X = train.drop(drop_cols, axis=1)
y = train['project_is_approved']

#del train

In [125]:
feature_names = list(X.columns)

#### Model Generation

In [39]:
# Build the model
cnt = 0
p_buf = []
n_splits = 5
n_repeats = 1
kf = RepeatedKFold(
    n_splits=n_splits, 
    n_repeats=n_repeats, 
    random_state=42)
auc_buf = []   

for train_index, valid_index in kf.split(X):
    print('Fold {}/{}'.format(cnt + 1, n_splits))
    params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'max_depth': 14,
        'num_leaves': 31,
        'learning_rate': 0.025,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.85,
        'bagging_freq': 5,
        'verbose': 0,
        'num_threads': 1,
        'lambda_l2': 1.0,
        'min_gain_to_split': 0,
    }  

    lgb_train = lgb.Dataset(
        X.loc[train_index], 
        y.loc[train_index], 
        feature_name=feature_names,
        )
    lgb_train.raw_data = None

    lgb_valid = lgb.Dataset(
        X.loc[valid_index], 
        y.loc[valid_index],
        )
    lgb_valid.raw_data = None

    model = lgb.train(
        params,
        lgb_train,
        num_boost_round=10000,
        valid_sets=[lgb_train, lgb_valid],
        early_stopping_rounds=100,
        verbose_eval=10,
    )

    if cnt == 0:
        importance = model.feature_importance()
        model_fnames = model.feature_name()
        tuples = sorted(zip(model_fnames, importance), key=lambda x: x[1])[::-1]
        tuples = [x for x in tuples if x[1] > 0]
        print('Important features:')
        for i in range(60):
            if i < len(tuples):
                print(tuples[i])
            else:
                break
gc.collect

Fold 1/5
Training until validation scores don't improve for 100 rounds.
[10]	training's auc: 0.856041	valid_1's auc: 0.605613
[20]	training's auc: 0.88737	valid_1's auc: 0.615679
[30]	training's auc: 0.903476	valid_1's auc: 0.610355
[40]	training's auc: 0.916518	valid_1's auc: 0.617111
[50]	training's auc: 0.934648	valid_1's auc: 0.622466
[60]	training's auc: 0.941806	valid_1's auc: 0.631749
[70]	training's auc: 0.951023	valid_1's auc: 0.640014
[80]	training's auc: 0.959157	valid_1's auc: 0.641742
[90]	training's auc: 0.964161	valid_1's auc: 0.644127
[100]	training's auc: 0.969761	valid_1's auc: 0.650547
[110]	training's auc: 0.978131	valid_1's auc: 0.654089
[120]	training's auc: 0.983334	valid_1's auc: 0.655609
[130]	training's auc: 0.986837	valid_1's auc: 0.653865
[140]	training's auc: 0.989999	valid_1's auc: 0.659135
[150]	training's auc: 0.992448	valid_1's auc: 0.656396
[160]	training's auc: 0.994229	valid_1's auc: 0.657607
[170]	training's auc: 0.995765	valid_1's auc: 0.658626
[18

Training until validation scores don't improve for 100 rounds.
[10]	training's auc: 0.838636	valid_1's auc: 0.632467
[20]	training's auc: 0.879459	valid_1's auc: 0.651805
[30]	training's auc: 0.895956	valid_1's auc: 0.66512
[40]	training's auc: 0.907966	valid_1's auc: 0.6633
[50]	training's auc: 0.919391	valid_1's auc: 0.656218
[60]	training's auc: 0.93479	valid_1's auc: 0.668411
[70]	training's auc: 0.941903	valid_1's auc: 0.675588
[80]	training's auc: 0.951936	valid_1's auc: 0.682924
[90]	training's auc: 0.957851	valid_1's auc: 0.684907
[100]	training's auc: 0.967418	valid_1's auc: 0.679792
[110]	training's auc: 0.976567	valid_1's auc: 0.681449
[120]	training's auc: 0.982281	valid_1's auc: 0.682654
[130]	training's auc: 0.987172	valid_1's auc: 0.686755
[140]	training's auc: 0.990231	valid_1's auc: 0.683614
[150]	training's auc: 0.993206	valid_1's auc: 0.686104
[160]	training's auc: 0.995296	valid_1's auc: 0.683186
[170]	training's auc: 0.996745	valid_1's auc: 0.679847
[180]	training'

Training until validation scores don't improve for 100 rounds.
[10]	training's auc: 0.859096	valid_1's auc: 0.633427
[20]	training's auc: 0.894185	valid_1's auc: 0.624954
[30]	training's auc: 0.913609	valid_1's auc: 0.634069
[40]	training's auc: 0.929638	valid_1's auc: 0.642123
[50]	training's auc: 0.938863	valid_1's auc: 0.636613
[60]	training's auc: 0.948437	valid_1's auc: 0.64098
[70]	training's auc: 0.953015	valid_1's auc: 0.64578
[80]	training's auc: 0.962511	valid_1's auc: 0.639241
[90]	training's auc: 0.969996	valid_1's auc: 0.64559
[100]	training's auc: 0.976122	valid_1's auc: 0.648097
[110]	training's auc: 0.981385	valid_1's auc: 0.646479
[120]	training's auc: 0.986122	valid_1's auc: 0.650914
[130]	training's auc: 0.990099	valid_1's auc: 0.650413
[140]	training's auc: 0.993029	valid_1's auc: 0.65257
[150]	training's auc: 0.994878	valid_1's auc: 0.652935
[160]	training's auc: 0.99623	valid_1's auc: 0.653633
[170]	training's auc: 0.997288	valid_1's auc: 0.65437
[180]	training's 

<function gc.collect>

In [92]:
doc = 0
feature_index = tfidf_matrix[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])

TypeError: 'TfidfVectorizer' object is not subscriptable

In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer
idf = tfidf.idf_
print(dict(zip(tfidf.get_feature_names(), idf)))

{'000': 6.221556305414575, '10': 5.110673924154651, '100': 3.972215726939341, '1000': 7.032486521630903, '10th': 7.032486521630903, '11': 6.116195789756748, '12': 4.953044979951068, '12th': 6.521660897864913, '13': 6.15009734143243, '14': 5.96204510992949, '15': 5.625572873308277, '150': 6.684179827362688, '16': 6.0208856099524235, '17': 6.426350718060588, '18': 5.456950160872484, '180': 6.952443813957367, '19': 6.339339341070958, '1st': 5.160684344729312, '20': 4.807862970106569, '200': 6.472870733695481, '2015': 6.952443813957367, '2016': 6.083405966933757, '2017': 6.809342970316694, '21': 5.906475258774679, '21st': 4.772461043055653, '22': 5.733203537500643, '23': 6.0208856099524235, '24': 5.4398557275131845, '25': 5.0514850527643205, '26': 6.744804449179123, '27': 6.381898955489754, '28': 6.051657268619177, '2nd': 4.853954077306836, '30': 5.186659831132572, '300': 6.744804449179123, '31': 6.878335841803645, '32': 6.952443813957367, '35': 6.684179827362688, '36': 6.952443813957367, 

In [40]:
auc_mean = np.mean(auc_buf)
auc_std = np.std(auc_buf)
print('AUC = {:.6f} +/- {:.6f}'.format(auc_mean, auc_std))

AUC = nan +/- nan


/Users/Nahel/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/Nahel/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/Nahel/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/Nahel/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/Nahel/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### NN

#### Params

In [ ]:
#Target
y = df[target].values[:182080]

len_cat = train_cat.shape[1]
len_num = train_num.shape[1]
len_text = train_text.shape[1]

size_embedding = 5000

#### Eval metrics

In [ ]:
# AUC for a binary classifier
def auc(y_true, y_pred):   
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)    
    return FP/N
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)    
    return TP/P

#### Architectures

In [ ]:
# categorical NN 
inputs1 = Input(shape=(len_cat,))
dense_cat_1 = Dense(256, activation='relu')(inputs1)
dense_cat_2 = Dense(128, activation='relu')(dense_cat_1)
dense_cat_3 = Dense(64, activation='relu')(dense_cat_2)
dense_cat_4 = Dense(32, activation='relu')(dense_cat_3)
final_cat = Dense(32, activation='relu')(dense_cat_4)

In [ ]:
# num channel
inputs2 = Input(shape=(len_num,))
dense_num_1 = Dense(256, activation='relu')(inputs2)
dense_num_2 = Dense(128, activation='relu')(dense_num_1)
dense_num_3 = Dense(64, activation='relu')(dense_num_2)
dense_num_4 = Dense(32, activation='relu')(dense_num_3)
final_num = Dense(32, activation='relu')(dense_num_4)

In [ ]:
# text chanel
inputs3 = Input(shape=(len_text,))
embedding3 = Embedding(size_embedding, 36)(inputs3)
conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
drop3 = Dropout(0.1)(conv3)
pool3 = MaxPooling1D(pool_size=2)(drop3)
final_text = Flatten()(pool3)

#### Evaluating individual Models

In [ ]:
#individual Model Tester
mod = Model(inputs1, Dense(1, activation='sigmoid')(final_cat))

In [ ]:
mod.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', auc])

In [ ]:
batch_size = 100
mod.fit(train_cat, y, batch_size = batch_size,
        epochs=3, validation_split=0.2)

#### Evaluating MultiChannel Model

In [ ]:
# merge
merged = concatenate([final_cat, final_num, final_text])

# Multi NN
dense1 = Dense(200, activation='relu')(merged)
dense2 = Dense(20, activation='relu')(dense1)
outputs = Dense(1, activation='sigmoid')(dense2)
model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)

In [ ]:
# Compile
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', auc])

In [ ]:
# Fitting
batch_size = 100
model.fit([train_cat, train_num, train_text], y, 
          batch_size=batch_size, epochs=2, validation_split=0.2)

# Submitting to Kaggle

In [ ]:
#2.5. Submitting
submit = model.predict([test_cat, test_num, test_text], 
                       batch_size=batch_size,verbose=1)

submission = pd.read_csv('./input/sample_submission.csv')
submission['project_is_approved'] = submit
submission.to_csv('pred.csv', index=False)

In [ ]:
#Non-multichannel type of NN
train_all = np.hstack((train_cat, train_real, train_text))
train_all.shape

In [ ]:
model2 = Sequential()
model2.add(Dense(256, input_shape=(trn_all.shape[1],), activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', auc])
batch_size = 2000
model2.fit(train_all, y, 
           batch_size=batch_size, epochs=3, 
           validation_split=0.2)

Conclusion
Certainly, computing power of Kaggle's kernel doesn't allow to build more sophisticated models, but in practice we may experiment with NN with multichannel input to achieve better results. Finally, NN with multichannel input are more flexible and let you work with different types of data.

Links
<a href = "https://keras.io" > Keras Documentation </a>
<a href = "https://machinelearningmastery.com/develop-n-gram-multichannel-convolutional-neural-network-sentiment-analysis/" >How to Develop an N-gram Multichannel Convolutional Neural Network for Sentiment Analysis </a>
<a href = https://towardsdatascience.com/neural-network-architectures-156e5bad51ba> Neural Network Architectures </a>